In [3]:
import pandas as pd
import numpy as np
import re, nltk, spacy, gensim


# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

csv = 'old_next_clean.csv'
issues_df = pd.read_csv(csv, encoding='latin1')

ModuleNotFoundError: No module named 'pyLDAvis'

In [2]:
issues_df

,Unnamed: 0,project_id,business_unit_x,project_id_classic,name_x,status_x,files,issues,checklists,equipment,...,city,state_or_province,postal_code,country,timezone,construction_type,contract_type,business_unit_id,last_sign_in,created_at_y
0,0,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,active,177,476,0,0,...,McCalla,AL,35111.0,US,America/Chicago,New Construction,Design-Bid-Build,39f43f7d-1eee-11e3-89a6-02cc3afa57bd,2020-01-20 17:42:48,2016-08-09 16:41:14.956157
1,1,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,active,177,476,0,0,...,McCalla,AL,35111.0,US,America/Chicago,New Construction,Design-Bid-Build,39f43f7d-1eee-11e3-89a6-02cc3afa57bd,2020-01-20 17:42:48,2016-08-09 16:41:14.956157
2,2,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,active,177,476,0,0,...,McCalla,AL,35111.0,US,America/Chicago,New Construction,Design-Bid-Build,39f43f7d-1eee-11e3-89a6-02cc3afa57bd,2020-01-20 17:42:48,2016-08-09 16:41:14.956157
3,3,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,active,177,476,0,0,...,McCalla,AL,35111.0,US,America/Chicago,New Construction,Design-Bid-Build,39f43f7d-1eee-11e3-89a6-02cc3afa57bd,2020-01-20 17:42:48,2016-08-09 16:41:14.956157
4,4,01fa4f3a-9969-4069-a5b3-72fb630a0143,"Clayco>Distribution, Manufacturing, Process, F...",6fbfddc0-b8bb-4b27-be7f-751a21c701d0,Gestamp,active,177,476,0,0,...,McCalla,AL,35111.0,US,America/Chicago,New Construction,Design-Bid-Build,39f43f7d-1eee-11e3-89a6-02cc3afa57bd,2020-01-20 17:42:48,2016-08-09 16:41:14.956157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101600,101600,b3ed4fac-47bb-43fb-bdd3-3973af5cd19a,Clayco>Institutional,6f732c19-f797-4bc8-bfdd-d3ec7d0e5283,10102939 - Memorial Stadium JV,active,552,6120,0,0,...,Champaign,IL,61820.0,US,America/Chicago,Renovation,Design-Bid-Build,03dc88ad-7eb0-48f5-8e12-859300f2a6c8,2020-03-26 13:01:25.472,2017-03-30 14:19:51.49377
101601,101601,b3ed4fac-47bb-43fb-bdd3-3973af5cd19a,Clayco>Institutional,6f732c19-f797-4bc8-bfdd-d3ec7d0e5283,10102939 - Memorial Stadium JV,active,552,6120,0,0,...,Champaign,IL,61820.0,US,America/Chicago,Renovation,Design-Bid-Build,03dc88ad-7eb0-48f5-8e12-859300f2a6c8,2020-03-26 13:01:25.472,2017-03-30 14:19:51.49377
101602,101602,b3ed4fac-47bb-43fb-bdd3-3973af5cd19a,Clayco>Institutional,6f732c19-f797-4bc8-bfdd-d3ec7d0e5283,10102939 - Memorial Stadium JV,active,552,6120,0,0,...,Champaign,IL,61820.0,US,America/Chicago,Renovation,Design-Bid-Build,03dc88ad-7eb0-48f5-8e12-859300f2a6c8,2020-03-26 13:01:25.472,2017-03-30 14:19:51.49377
101603,101603,b3ed4fac-47bb-43fb-bdd3-3973af5cd19a,Clayco>Institutional,6f732c19-f797-4bc8-bfdd-d3ec7d0e5283,10102939 - Memorial Stadium JV,active,552,6120,0,0,...,Champaign,IL,61820.0,US,America/Chicago,Renovation,Design-Bid-Build,03dc88ad-7eb0-48f5-8e12-859300f2a6c8,2020-03-26 13:01:25.472,2017-03-30 14:19:51.49377


In [3]:
issues_df = issues_df.dropna(subset=['description'])

In [4]:
# Convert to list
data = issues_df.description.values.tolist()
# Remove Emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub(r"\'", "", sent) for sent in data]
pprint(data[:1])

['Remove nails around dock angles. ']


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['remove', 'nails', 'around', 'dock', 'angles']]


In [6]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

In [8]:
# Initialize spacy ‘en’ model, keeping only tagger component (for efficiency)
# Run in terminal: python -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'VERB']) #select noun and verb
print(data_lemmatized[:2])

['remove nail dock angle', 'grind chamfer corner tilt panel corner parapet corner trucker lounge parapet detail']


In [10]:
vectorizer = CountVectorizer(analyzer='word', min_df=10,stop_words='english', lowercase=True, token_pattern='[a-zA-Z0-9]{3,}', max_features = 5000) 
data_vectorized = vectorizer.fit_transform(data_lemmatized)           

In [14]:
lda_model = LatentDirichletAllocation(n_components=10, max_iter = 10, learning_method = 'online', random_state =100, batch_size = 50, evaluate_every = -1, n_jobs = -1)

In [15]:
lda_output = lda_model.fit_transform(data_vectorized)

In [16]:
print(lda_model)

LatentDirichletAllocation(batch_size=50, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=100, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [17]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -2801852.3105766876
Perplexity:  551.6868694007248
{'batch_size': 50,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [18]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)

C:\Users\mrabier\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='online',
                                                 learning_offset=50.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=5,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1, random_state=0,
                                                 topic_word_prior=None,
                                                 total_samples=1000000.0,
             

In [19]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 10}
Best Log Likelihood Score:  -953094.314135732
Model Perplexity:  459.55962131650557


In [25]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ['Topic' + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ['Doc' + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
def color_green(val):
 color = 'green' if val > .1 else 'black'
 return 'color: {col}'.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return 'font-weight: {weight}'.format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.82,9
Doc1,0.01,0.01,0.01,0.09,0.01,0.01,0.01,0.85,0.01,0.01,7
Doc2,0.68,0.02,0.02,0.02,0.02,0.02,0.18,0.02,0.02,0.02,0
Doc3,0.82,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0
Doc4,0.7,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0
Doc5,0.03,0.03,0.03,0.77,0.03,0.03,0.03,0.03,0.03,0.03,3
Doc6,0.03,0.03,0.25,0.03,0.03,0.03,0.03,0.03,0.55,0.03,8
Doc7,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0
Doc8,0.02,0.02,0.02,0.02,0.02,0.5,0.02,0.34,0.02,0.02,5
Doc9,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.41,0.33,8


In [26]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

,abandon,ability,accent,accept,acceptance,access,accessibility,accessory,accommodate,accord,...,worker,working,wrap,write,writing,wshp,yard,year,zip,zone
Topic0,0.100392,0.100271,0.100324,0.100237,0.100255,0.100863,0.100280,88.930269,0.100288,0.100275,...,0.100336,0.100285,0.100437,0.100734,0.100244,0.100296,0.100279,0.100280,0.100812,0.194651
Topic1,0.100371,0.218933,0.100244,2.350308,0.141799,0.105772,0.100287,0.100284,1.009866,2.252476,...,1.809534,0.100395,138.790423,105.330263,0.100290,10.662648,36.309020,0.105532,0.101754,0.100487
Topic2,0.100372,18.276239,0.100296,13.384524,0.100383,161.720688,0.100346,0.100390,13.213368,0.125845,...,179.310653,20.297401,0.123540,10.249103,16.585855,0.100350,0.129007,30.275007,0.101576,143.593485
Topic3,0.102572,0.100258,44.635793,0.118018,0.100275,0.101662,0.100308,2.537376,0.997334,1.931370,...,0.100384,0.100350,0.100899,0.104676,0.127373,0.100299,0.100272,0.144403,0.100311,0.102535
Topic4,43.059017,0.113154,0.100365,0.119579,15.943735,0.101036,0.100269,0.100440,35.560233,21.943305,...,125.985728,0.100386,3.647174,0.101840,0.100218,0.101972,0.116796,0.100812,31.545065,0.100478


In [27]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,install,repair,cover,door,plate,light,switch,outlet,hardware,base,device,faceplate,opening,block,dent
Topic 1,water,installation,provide,pipe,stop,valve,label,piping,air,work,support,remain,degree,supply,panel
Topic 2,require,level,sprinkler,head,grid,issue,use,stair,item,control,duct,cabinet,crack,photo,attach
Topic 3,wall,room,seal,caulk,column,panel,trim,gap,penetration,conduit,sealant,end,cut,location,cap
Topic 4,instal,drywall,insulation,complete,equipment,place,steel,make,space,drawing,mud,beam,tool,deck,break
Topic 5,paint,touch,tape,remove,indicate,sticker,fixture,mark,door,protection,edge,access,note,scratch,close
Topic 6,ceiling,tile,miss,unit,finish,shade,smoke,bathroom,detector,carpet,closet,sensor,wire,bedroom,damage
Topic 7,frame,floor,door,patch,base,corner,hole,shower,remove,rail,location,receptacle,area,concrete,screw
Topic 8,window,door,replace,adjust,glass,sill,damage,box,need,screen,gasket,seat,sign,latch,office
Topic 9,need,clean,area,remove,work,sink,fix,floor,drain,material,surface,nail,debris,roof,building
